<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/algo_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from pandas_datareader import data as pdr

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd

folder_path = '/content/gdrive/MyDrive/csv/bank_ nifty_csv'  # Corrected folder path to remove space
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames read from each CSV
dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# You can optionally reset the index if you prefer
# combined_df.reset_index(drop=True, inplace=True)

# Optionally, export the combined DataFrame to a single CSV file
combined_file_path = '/content/gdrive/MyDrive/csv/combined_bank_nifty.csv'
combined_df.to_csv(combined_file_path, index=False)

print("Combined CSV file saved successfully.")


In [3]:
import os
import pandas as pd

folder_path = '/content/gdrive/MyDrive/csv/bank_ nifty_csv'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path,nrows=100)
df['datetime'] = pd.to_datetime(df['expiry'] + ' ' + df['time'])
df = df.sort_values('datetime')
df.head()
#print(df.tail())


<ipython-input-3-1922dcff32bd>:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['expiry'] + ' ' + df['time'])


,time,symbol,open,high,low,close,volume,oi,expiry,strike,instrument_type,datetime
4,09:17:00,BANKNIFTY11NOV10000CE,146.1,147.0,146.00,146.0,50,40450,24-11-2011,10000,CE,2011-11-24 09:17:00
5,09:18:00,BANKNIFTY11NOV10000CE,146.0,146.0,136.05,146.0,100,40450,24-11-2011,10000,CE,2011-11-24 09:18:00
6,09:19:00,BANKNIFTY11NOV10000CE,146.0,150.0,146.00,150.0,50,40500,24-11-2011,10000,CE,2011-11-24 09:19:00
7,09:20:00,BANKNIFTY11NOV10000CE,150.0,150.0,140.00,140.0,100,40500,24-11-2011,10000,CE,2011-11-24 09:20:00
8,09:21:00,BANKNIFTY11NOV10000CE,140.0,140.0,137.00,138.0,100,40500,24-11-2011,10000,CE,2011-11-24 09:21:00


In [5]:
df.shape
#df.tail()

(100, 12)

In [31]:
df['datetime'] = pd.to_datetime(df['expiry'] + ' ' + df['time'])
df = df.sort_values('datetime')

# Resample based on 'datetime' without setting it as the index
df_resampled = df.resample('1T', on='datetime').ffill()

<ipython-input-31-ca75f74860ea>:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['expiry'] + ' ' + df['time'])


ValueError: Upsampling from level= or on= selection is not supported, use .set_index(...) to explicitly set index to datetime-like

In [24]:
df_5min = df.resample('5T').agg({
    'open': 'first','high': 'max','low': 'min','close': 'last','volume': 'sum','oi':'sum'
})

In [ ]:
'''df['time'] = pd.to_datetime(df['time'])
    df_filtered = df[df['time'].dt.year > 2018]'''

In [4]:
import os

# Path to the folder containing CSV files
folder_path = '/content/gdrive/MyDrive/csv/bank_ nifty_csv'

# List all files in the folder
files = os.listdir(folder_path)

# Count the number of CSV files
csv_count = sum(1 for file in files if file.endswith('.csv'))

print("Number of CSV files in the folder:", csv_count)

Number of CSV files in the folder: 3009


Intraday PCR (Put Call Ratio):

In [ ]:
put_volume = data[data['instrument_type'] == 'put']['volume'].sum()
call_volume = data[data['instrument_type'] == 'call']['volume'].sum()

intraday_pcr = put_volume / call_volume

Cumulative PCR:

In [ ]:
atm_strike = data[data['strike'] == ATM_strike]
cumulative_pcr = atm_strike['volume'].sum()

Most Active Strikes:

In [ ]:
strike_volume = data.groupby('strike')['volume'].sum()

most_active_strikes = strike_volume.nlargest(6)

Max Pain:

In [ ]:
call_oi = data[data['instrument_type'] == 'call'].groupby('strike')['oi'].sum()
put_oi = data[data['instrument_type'] == 'put'].groupby('strike')['oi'].sum()

max_pain_strike = (call_oi.idxmax() + put_oi.idxmax()) / 2

VWAP (Volume Weighted Average Price):

In [ ]:
vwap = data['vwap']
# Calculate VWAP
data['vwap'] = (data['volume'] * (data['high'] + data['low'] + data['close']) / 3).cumsum() / data['volume'].cumsum()


MACD (Moving Average Convergence Divergence):

In [ ]:
# Calculate the short-term Exponential Moving Average (EMA)
short_ema = data['close'].ewm(span=12, adjust=False).mean()

# Calculate the long-term Exponential Moving Average (EMA)
long_ema = data['close'].ewm(span=26, adjust=False).mean()

# Calculate the MACD line
data['macd'] = short_ema - long_ema

# Calculate the signal line
data['signal_line'] = data['macd'].ewm(span=9, adjust=False).mean()


POC (Point of Control):
Calculate the price level with the highest trading volume within the defined time frame:


In [ ]:
poc = data['volume'].idxmax()

Ghost Zones:
Calculate the range above VAH and below VAL:

In [ ]:
ghost_zones = (vah - val) / val * 100

Local Maxima and Minima:

In [ ]:
local_maxima = data['high'].max()
local_minima = data['low'].min()

Price Opening:

Determine if the opening price is above VAH, within VAH and VAL, or below VAL:

In [ ]:
opening_price = data.iloc[0]['open']

if opening_price > vah:
    price_opening = "Above VAH"
elif val < opening_price <= vah:
    price_opening = "Within VAH and VAL"
else:
    price_opening = "Below VAL"

**PLOT GRAPH TO IMPRESS**

In [ ]:
'''import zipfile
import pandas as pd
from sqlalchemy import create_engine

# Path to the ZIP file
zip_file_path = "/content/gdrive/MyDrive/Copy of BANKNIFTY OPT.zip"

# Extract all files from the ZIP archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gdrive/MyDrive/Copy of BANKNIFTY OPT")

# Assuming you want to read both DB files, BANKNIFTY_FUT.db and BANKNIFTY_OPT.db
# If you want to read just one file, specify the file path accordingly

# Create SQLAlchemy engine
engine = create_engine('sqlite:////content/gdrive/MyDrive/Copy of BANKNIFTY OPT/BANKNIFTY_FUT.db')

# Read data from the database table into a DataFrame
data_fut = pd.read_sql("SELECT * FROM BANKNIFTY_FUT", con=engine)

# Repeat the same process for the other database file
engine = create_engine('sqlite:////content/gdrive/MyDrive/Copy of BANKNIFTY OPT/BANKNIFTY_OPT.db')
data_opt = pd.read_sql("SELECT * FROM BANKNIFTY_OPT", con=engine)

# Now you can work with the extracted data
'''

OperationalError: (sqlite3.OperationalError) no such table: BANKNIFTY_FUT
[SQL: SELECT * FROM BANKNIFTY_FUT]
(Background on this error at: https://sqlalche.me/e/20/e3q8)